In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(columns = ['EIN','NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Optimization Attempt 1

In [3]:
# Analyze Income_amount
application_df["INCOME_AMT"].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [4]:
# Drop "INCOME_AMT" income groupings are not standardized, majority of cases are 0, could skew average
application_df = application_df.drop(columns = "INCOME_AMT", axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,N,142590,1


In [5]:
# APPLICATION_TYPE value counts for binning
applic_counts = application_df.APPLICATION_TYPE.value_counts()
applic_counts

# Determine which values to replace if counts are less than ...?
replace_application = list(applic_counts[applic_counts<500].index)
# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# CLASSIFICATION value counts for binning
classif_counts = application_df.CLASSIFICATION.value_counts()

# Replace values <1500
replace_class = list(classif_counts[classif_counts<1500].index)
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
cat_list = application_df.dtypes[application_df.dtypes=="object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df = pd.DataFrame(enc.fit_transform(application_df[cat_list]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(cat_list)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [8]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(cat_list,1)
application_df.head()

C:\Users\Big Pappa J\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns = "IS_SUCCESSFUL",axis=1).values
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 60

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 100)               3500      
                                                                 
 dense_21 (Dense)            (None, 80)                8080      
                                                                 
 dense_22 (Dense)            (None, 60)                4860      
                                                                 
 dense_23 (Dense)            (None, 1)                 61        
                                                                 
Total params: 16,501
Trainable params: 16,501
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# import dependencise for checkpoints
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Create Checkpoints and file names
os.makedirs("checkpoints_opt1/",exist_ok=True)
checkpoint_path = "checkpoints_opt1/weights.{epoch:02d}.hdf5"

# Create callback to save model's weight every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn1.fit(X_train_scaled,y_train, epochs=200,callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 1s 906us/step - loss: 0.5473 - accuracy: 0.7358
Epoch 2/200
804/804 [==============================] - 1s 937us/step - loss: 0.5470 - accuracy: 0.7355
Epoch 3/200
804/804 [==============================] - 1s 940us/step - loss: 0.5463 - accuracy: 0.7353
Epoch 4/200
804/804 [==============================] - 1s 929us/step - loss: 0.5468 - accuracy: 0.7361
Epoch 5/200
778/804 [============================>.] - ETA: 0s - loss: 0.5459 - accuracy: 0.7358
Epoch 5: saving model to checkpoints_opt1\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5457 - accuracy: 0.7362
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7353
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7355
Epoch 8/200
804/804 [==============================] - 1s 981us/step - loss: 0.5459 - accuracy: 0.7352
Epoch 9/200
804/804 [==================

804/804 [==============================] - 1s 979us/step - loss: 0.5485 - accuracy: 0.7360
Epoch 63/200
804/804 [==============================] - 1s 961us/step - loss: 0.5467 - accuracy: 0.7368
Epoch 64/200
804/804 [==============================] - 1s 938us/step - loss: 0.5453 - accuracy: 0.7368
Epoch 65/200
755/804 [===========================>..] - ETA: 0s - loss: 0.5450 - accuracy: 0.7363
Epoch 65: saving model to checkpoints_opt1\weights.65.hdf5
804/804 [==============================] - 1s 985us/step - loss: 0.5454 - accuracy: 0.7360
Epoch 66/200
804/804 [==============================] - 1s 904us/step - loss: 0.5451 - accuracy: 0.7372
Epoch 67/200
804/804 [==============================] - 1s 916us/step - loss: 0.5454 - accuracy: 0.7367
Epoch 68/200
804/804 [==============================] - 1s 900us/step - loss: 0.5454 - accuracy: 0.7365
Epoch 69/200
804/804 [==============================] - 1s 912us/step - loss: 0.5451 - accuracy: 0.7370
Epoch 70/200
750/804 [===============

Epoch 124/200
804/804 [==============================] - 1s 945us/step - loss: 0.5446 - accuracy: 0.7369
Epoch 125/200
761/804 [===========================>..] - ETA: 0s - loss: 0.5442 - accuracy: 0.7370
Epoch 125: saving model to checkpoints_opt1\weights.125.hdf5
804/804 [==============================] - 1s 946us/step - loss: 0.5444 - accuracy: 0.7369
Epoch 126/200
804/804 [==============================] - 1s 939us/step - loss: 0.5446 - accuracy: 0.7364
Epoch 127/200
804/804 [==============================] - 1s 971us/step - loss: 0.5446 - accuracy: 0.7371
Epoch 128/200
804/804 [==============================] - 1s 922us/step - loss: 0.5445 - accuracy: 0.7375
Epoch 129/200
804/804 [==============================] - 1s 946us/step - loss: 0.5450 - accuracy: 0.7358
Epoch 130/200
767/804 [===========================>..] - ETA: 0s - loss: 0.5446 - accuracy: 0.7364
Epoch 130: saving model to checkpoints_opt1\weights.130.hdf5
804/804 [==============================] - 1s 945us/step - loss:

756/804 [===========================>..] - ETA: 0s - loss: 0.5445 - accuracy: 0.7381
Epoch 185: saving model to checkpoints_opt1\weights.185.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5456 - accuracy: 0.7368
Epoch 186/200
804/804 [==============================] - 1s 978us/step - loss: 0.5447 - accuracy: 0.7367
Epoch 187/200
804/804 [==============================] - 1s 994us/step - loss: 0.5444 - accuracy: 0.7376
Epoch 188/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5461 - accuracy: 0.7362
Epoch 189/200
804/804 [==============================] - 1s 989us/step - loss: 0.5441 - accuracy: 0.7363
Epoch 190/200
787/804 [============================>.] - ETA: 0s - loss: 0.5425 - accuracy: 0.7381
Epoch 190: saving model to checkpoints_opt1\weights.190.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5440 - accuracy: 0.7367
Epoch 191/200
804/804 [==============================] - 1s 1000us/step - loss: 0.5440 - accuracy:

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5737 - accuracy: 0.7297 - 277ms/epoch - 1ms/step
Loss: 0.5737488865852356, Accuracy: 0.72967928647995


In [36]:
# Save Model 
nn1.save("AlphabetSoupCharity_optimzatio_v01.h5")